In [8]:
import pandas as pd
import numpy as np

#Read CSV dataset files from current directory into Numpy arrays
hospitalization_df = pd.read_csv('aggregated_cc_by.csv',sep = ',',low_memory=False)
search_trends_df = pd.read_csv('2020_US_weekly_symptoms_dataset.csv',sep = ',',low_memory=False)

#Isolate Hospitalization data pertaining to US States in Search Trends dataset
hospitalization_df=hospitalization_df[hospitalization_df['open_covid_region_code'].isin(search_trends_df['open_covid_region_code'].unique())]

#Sort Hospitalization dataset based on state then date
hospitalization_df=hospitalization_df.sort_values(by=['open_covid_region_code','date'])

#Remove data before 2020-03-09 in the Symptom Searches dataset in order to merge both datasets
#Note that 2020-03-09 is the first Monday which has data in both datasets for the US regions
search_trends_df=search_trends_df[search_trends_df['date'] >= "2020-03-09"]

#Remove data before 2020-03-09 in the Hospitalization dataset in order to merge both datasets
#Note that 2020-03-09 is the first Monday which has data in both datasets for the US regions
hospitalization_df=hospitalization_df[hospitalization_df['date'] >= "2020-03-09"]

#Remove data after 2020-09-27 (last Sunday in both datasets) from both datasets
#in order to merge both datasets on a weekly basis
hospitalization_df=hospitalization_df[hospitalization_df['date'] <= "2020-09-27"]
search_trends_df=search_trends_df[search_trends_df['date'] <= "2020-09-27"]

#Delete columns from Hospitalization and Search Trends data that contain only NaN values
hospitalization_df=hospitalization_df.dropna(how='all', axis=1)
search_trends_df=search_trends_df.dropna(how='all', axis=1)

#Isolate data for each Sunday of week (weekly cumulative total of Covid Hospitalizations is at the end of the week)
sunday_data=hospitalization_df[pd.to_datetime(hospitalization_df['date']).dt.dayofweek==6].copy(deep=True)

#Replace daily data in Hospitalization dataset with the Monday date of the corresponding week
hospitalization_df['date']=pd.to_datetime(hospitalization_df['date']) - pd.to_timedelta(pd.to_datetime(hospitalization_df['date']).dt.dayofweek, unit='d')

#Add up daily Covid cases to get weekly count
sum_weekly_data=hospitalization_df.groupby(['open_covid_region_code','date']).sum().reset_index()['hospitalized_new']
sunday_data=sunday_data.reset_index().assign(hospitalized_new=sum_weekly_data)
sunday_data['date']=pd.to_datetime(sunday_data['date']) - pd.to_timedelta(pd.to_datetime(sunday_data['date']).dt.dayofweek, unit='d')
hospitalization_df=sunday_data

#Merge data
merged_data_df = pd.concat([hospitalization_df.reset_index(drop=True),search_trends_df.reset_index(drop=True)],axis=1)

#Delete duplicate columns and unnecessary columns
merged_data_df = merged_data_df.loc[:,~merged_data_df.columns.duplicated()]
merged_data_df=merged_data_df.drop(['index','sub_region_1','open_covid_region_code','hospitalized_cumulative','country_region_code','country_region','sub_region_1_code'], axis=1)

#Remove columns with more than 35% NaN values
threshold=0.35
missings = merged_data_df.isna().sum()/merged_data_df.shape[0]
deleting = merged_data_df.columns[missings>threshold]
cols = merged_data_df.columns[missings<=threshold]
merged_data_df.drop(deleting,axis=1,inplace=True)

symptoms = merged_data_df.columns[3:]
filldf = pd.DataFrame(data=np.zeros(merged_data_df.shape),columns=merged_data_df.columns)
themin = np.inf

for state in merged_data_df['region_name'].unique():
    location = merged_data_df['region_name']==state, symptoms
    k = merged_data_df.loc[location]
    #Minimum accross all symptom trends in specific US region
    min2 = np.min(np.min(k))
    if min2<themin:
        #Minimum accross all symptom trends and US regions
        themin = min2
    filldf.loc[location] = min2

#Fill NaN values in each column with column minimum
merged_data_df.fillna(filldf,inplace=True)
if merged_data_df.isnull().values.any():
    merged_data_df.fillna(themin,inplace=True)   
    
#Delete states with 0 hospitalizations
states = merged_data_df['region_name'].unique()
threshold_on_0=0.8
for state in states:
    hospitalized_in_state = merged_data_df.loc[merged_data_df['region_name']==state,'hospitalized_new']
    hospitalized_count = len(hospitalized_in_state[hospitalized_in_state==0].values)
    if hospitalized_count>hospitalized_in_state.shape[0]*threshold_on_0:
        merged_data_df.drop(merged_data_df[merged_data_df['region_name']==state].index,axis=0,inplace=True) 
                
#We will take into account internet population
#because the data only makes sense in one state, it is comparable. If we want to 
#compare between states, we need to scale them.
internet_usage_df = pd.read_excel('internet_usage.xlsx')
internet_usage_df.drop(['total population', 'percentage of internet users'],axis=1, inplace=True)
states = merged_data_df['region_name'].unique()
internet_usage_df.rename(columns={ internet_usage_df.columns[0]: "states" }, inplace = True)
truncated_internet_usage_df = internet_usage_df.loc[internet_usage_df['states'].isin(states)]
cols = merged_data_df.columns[3:]
for state in truncated_internet_usage_df['states']:
    merged_data_df.loc[merged_data_df['region_name']==state, cols] *= float(truncated_internet_usage_df[truncated_internet_usage_df['states']==state]['No. internet users']/100000)

#States like Nebraska has a crazy increases in hospitalization_new at some points
#Here we remove those States from the dataset
for state in ['Nebraska','New Mexico','Rhode Island']:
    indexes = merged_data_df[merged_data_df['region_name']==state].index
    merged_data_df.drop(indexes,inplace=True)

#Reset indices
merged_data_df.reset_index(drop=True, inplace=True)

#Generate CSV file from dataframe
merged_data_df.to_csv('merged_data.csv', encoding='utf-8')

print(merged_data_df)






    region_name       date  hospitalized_new  symptom:Angular cheilitis  \
0        Hawaii 2020-03-09               0.0                 373.072786   
1        Hawaii 2020-03-16               0.0                 330.136609   
2        Hawaii 2020-03-23              12.0                 363.979065   
3        Hawaii 2020-03-30               7.0                 453.419840   
4        Hawaii 2020-04-06              25.0                 417.390287   
..          ...        ...               ...                        ...   
227     Wyoming 2020-08-24               8.0                   9.934225   
228     Wyoming 2020-08-31               4.0                   9.934225   
229     Wyoming 2020-09-07               9.0                  10.300463   
230     Wyoming 2020-09-14              15.0                   9.934225   
231     Wyoming 2020-09-21              19.0                   9.934225   

     symptom:Aphonia  symptom:Crackles  symptom:Dysautonomia  \
0         739.814500        363.518